In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [7]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [8]:
dataset.dtypes

User ID            int64
Age                int64
EstimatedSalary    int64
Purchased          int64
Gender_Male         bool
dtype: object

In [9]:
dataset=dataset.drop("User ID",axis=1)

In [10]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [11]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [12]:
indep = dataset[['Gender_Male', 'Age', 'EstimatedSalary']]
dep = dataset[['Purchased']]

In [13]:
indep.shape

(400, 3)

In [14]:
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [16]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


In [17]:
from sklearn.svm import SVC

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid={'kernel':['linear','rbf','poly','sigmoid'],
           'gamma':['auto','scale'],
           'C':[10,100,1000,2000]}

grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,cv=5,n_jobs=-1,scoring='f1_weighted')

grid.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


C:\Users\Arun\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000], 'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [21]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [22]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for the best parameter {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9100355779243318


In [23]:
print("The confustion Matrix:\n",cm)

The confustion Matrix:
 [[80  5]
 [ 7 42]]


In [24]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [25]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.9539015606242497)

In [26]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.026433,0.001718,0.016057,0.001548,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,17
1,0.023787,0.003505,0.018381,0.001547,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
2,0.025216,0.002878,0.016304,0.001344,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,9
3,0.016564,0.001779,0.014636,0.000908,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.762677,0.738916,0.655795,0.796284,0.766556,0.744045,0.047743,31
4,0.022899,0.002256,0.015071,0.002074,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,17
5,0.016740,0.003416,0.014130,0.001278,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
6,0.021652,0.001111,0.013898,0.001028,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,9
7,0.015561,0.003418,0.013652,0.002488,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.762677,0.738916,0.753180,0.778067,0.766556,0.759879,0.013172,25
8,0.067520,0.005177,0.014886,0.001062,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,17
9,0.023110,0.004393,0.013695,0.001426,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.870362,0.944161,0.943041,0.902367,0.034308,1


In [28]:
age_input=float(input('Age:'))
salary_input=float(input('EstimatedSalary'))
sex_male_input=int(input('Sex_Male 0 or 1:'))

Age: 30
EstimatedSalary 30000
Sex_Male 0 or 1: 1


In [29]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
